In [ ]:
import os
import intake
import pandas as pd
import xarray as xr

df = intake.open_csv('data/bird_migration/{species}.csv').read()

def fill_day(v):
    next_year = v.assign(day=v.day + v.day.max())
    last_year = v.assign(day=v.day - v.day.max())
    surrounding_years = pd.concat([last_year, v, next_year])
    filled = surrounding_years.assign(
        lat=surrounding_years.lat.interpolate(), 
        lon=surrounding_years.lon.interpolate())
    this_year = filled[filled.day.isin(v.day)]
    return this_year

df = pd.concat([fill_day(v) for k, v in df.groupby('species')])

colors = pd.read_csv('./assets/colormap.csv', header=None, names=['R', 'G', 'B'])
species_cmap = dict(zip(df.species.cat.categories, 
                        ['#{row.R:02x}{row.G:02x}{row.B:02x}'.format(row=row) 
                         for _, row in colors.iterrows()]))

data_url = 'http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/ncep/air.day.ltm.nc'

# I downloaded the file locally because I was hitting rate limits.
local_file = './data/air.day.ltm.nc'
if os.path.isfile(local_file):
    data_url = local_file

ds = xr.open_dataset(data_url)
ds = ds.rename(time='day').sel(level=1000)
ds['day'] = list(range(1,366))

## convert to F
ds = ds.assign(air_F = (ds['air'] - 273.15) * 9/5 + 32)

## Panel 

Panel provides the framework and interactivity to make dashboards that work in the notebook and can be deployed as standalone apps. Just to remind ourselves - we have our target dashboard.

<img src="./assets/target_dashboard.jpg" width=40%></img>

In [ ]:
import numpy as np

import hvplot.pandas
import hvplot.xarray

import holoviews as hv
import geoviews as gv
import geoviews.tile_sources as gts
import cartopy.crs as ccrs

from holoviews.streams import Selection1D, Params
import panel as pn

hv.extension('bokeh', width=90)
pn.extension()

### Make some widgets

One of the things that panel provides is an easy way to instantiate widgets that work both inside and outside the notebook. We'll set up a `Player` widget, a `MultiSelect` widget, and a `Toggle`.

In [ ]:
species = pn.widgets.MultiSelect(options=df.species.cat.categories.tolist(), size=10)
species

Select an item from the list above or shift + click or cmd/ctrl + click to select multiple species

In [ ]:
species.value

In [ ]:
day = pn.widgets.Player(value=1, start=1, end=365, loop_policy='loop', name='day', width=350, step=5)
day

In [ ]:
day.value

In [ ]:
toggle = pn.widgets.Toggle(name='Air Temperature Layer', active=True)
toggle

In [ ]:
toggle.active

Now we can capture the streams from these widgets so we can use them in our dynamic maps. 

In [ ]:
species_stream = Params(species, ['value'], rename={'value': 'species'})
day_stream = Params(day, ['value'], rename={'value': 'day'})
toggle_stream = Params(toggle, ['active'])

In [ ]:
def sanity_checker(species):
    return hv.Text(0.5, 0.5, '\n'.join(species))

hv.util.DynamicMap(sanity_checker, streams=[species_stream])

In [ ]:
species

We'll make a reset button to reset all these widgets. 

In [ ]:
def reset(arg=None):
    day_stream.update(value=1)
    species_stream.update(value=[])
    toggle_stream.update(active=True)
    
reset_button = pn.widgets.Button(name='Reset')
reset_button.param.watch(reset, 'clicks')
reset_button

**NOTE:** Click the button and then go back and look at the "sanity_checker"

### Constructing pieces

We'll start by setting up a map of bird locations grouped by day of year. This plot is the same as we've set up in prior notebooks.

In [ ]:
birds = df.hvplot.points('lon', 'lat', color='species', groupby='day', geo=True,
                         cmap=species_cmap, legend=False, size=100).options(tools=['tap', 'hover', 'box_select'], 
                                                                  width=500, height=600)

In [ ]:
tiles = gts.EsriImagery()
tiles.extents = df.lon.min(), df.lat.min(), df.lon.max(), df.lat.max()

In [ ]:
birds * tiles

Now we can take the player widget that we've defined above and use that one instead of the slider. 

In [ ]:
bird_dmap = birds.clone(streams=[day_stream])
row = pn.Row(bird_dmap * tiles, day)
row

That doesn't look quite how we want it. So we can inspect the structure of the panel object.

In [ ]:
print(row)

Each of the items has an index, so we can access the components individually. Note that the components are still linked (try wiggling the slider).

In [ ]:
row[0][1]

In [ ]:
pn.Row(row[0][0], row[1])

### Adding another layer

Let's add the temperature data now. To speed things up we can subset the air temperature layer to the region of interest, and then persist that in memory.

In [ ]:
extents = df.lon.min(), df.lon.max(), df.lat.min(), df.lat.max()
extents

In [ ]:
360+extents[0], 360+extents[1]

One tricky thing is figuring out the right order for the slices. We'll do it by inspection, but there is probably a more clever way.

In [ ]:
ROI = ds.sel(lon=slice(205, 310), lat=slice(75, -55)).persist()
ROI

In [ ]:
p = ROI.hvplot.quadmesh('lon', 'lat', 'air_F', groupby='day', geo=True)
p

You'll notice that as you slide around the days, the colorbar hops around to accommodate the range of cell values. We can fix that by taking a look at the min and max and using those values to set the allowable range for air temperature.

In [ ]:
ds.air_F.min().item(), ds.air_F.max().item()

So we'll do some plot tweaking and then clone the air plot to accept the day_stream as we did above for birds:

In [ ]:
grouped_air = p.options(height=600, width=500, tools=[]).redim.range(air_F=(-20, 100))
air_dmap = grouped_air.clone(streams=[day_stream])

The last step for the temperature layer is to add in the toggle. For that we'll create a function that accept the dynamic map and the active stream from the toggle. Then we'll create a new dynamic map that wraps our air_dmap. 

In [ ]:
def toggle_temp(layer, active=True):
    return layer.options(fill_alpha=int(active))

temp_layer = hv.util.Dynamic(air_dmap, operation=toggle_temp, streams=[toggle_stream])

row = pn.Row(pn.widgets.WidgetBox(toggle, day, width=450), pn.Row(tiles * temp_layer * gv.feature.coastline * bird_dmap)[0][0])
row

In [ ]:
print(row)

It's kind of hard to see those birds so let's set the line color to white and add another toggle to turn that on and off.

In [ ]:
highlight = pn.widgets.Toggle(name='Highlight Birds', active=False)
highlight_stream = Params(highlight, ['active'])

def do_highlight(points, active=True):
    return points.options(line_alpha=(0.5 if active else 0), selection_line_alpha=active)

bird_dmap = hv.util.Dynamic(bird_dmap.options(line_color='white'), operation=do_highlight, streams=[highlight_stream])

In [ ]:
pn.Row(highlight, bird_dmap * tiles)

### Doing a little more computation
One of the things that we'd like to display on our dashboard is bird speed. We can calculate the speed for each bird for each day using `pyproj`.

In [ ]:
import pyproj
import numpy as np

g = pyproj.Geod(ellps='WGS84')

def calculate_speed(v):
    today_lat = v['lat'].values
    today_lon = v['lon'].values
    tomorrow_lat = np.append(v['lat'][1:].values, v['lat'][0])
    tomorrow_lon = np.append(v['lon'][1:].values, v['lon'][0])
    _, _, dist = g.inv(today_lon, today_lat, tomorrow_lon, tomorrow_lat)
    return v.assign(speed=dist/1000.)

df = pd.concat([calculate_speed(v) for k, v in df.groupby('species')])
df.head()

### Defining functions

A more common way to link streams with plots is using a function that accepts some values from streams as input and returns some holoviews object. Here we will define a function that plots lat vs day for all species or a select list of species depending on the input.

In [ ]:
def timeseries(species=None, y='lat'):
    data = df[df.species.isin(species)] if species else df
    plots = [
        (data.groupby(['day', 'species'], observed=True)[y]
            .mean()
            .groupby('day').agg([np.min, np.max])
            .hvplot.area('day', 'amin', 'amax', alpha=0.2, fields={'amin': y}))]
    if not species or len(species) > 7:
        plots.append(data.groupby('day')[y].mean().hvplot().relabel('mean'))
    else:
        gb = data.groupby('species', observed=True)
        plots.extend([v.hvplot('day', y, color=species_cmap[k]).relabel(k) for k, v in gb])
    return hv.Overlay(plots).options(width=900, height=250, toolbar='below', legend_position='right', legend_offset=(20, 0), label_width=150)

In [ ]:
timeseries()

In [ ]:
timeseries(['Veery', 'Connecticut_Warbler'], 'speed')

Now that we have a sense of how the function works, we can set up a `holoviews.DynamicMap` with species and day as the streams and our function as the callable. 

In [ ]:
ts_lat = hv.DynamicMap(lambda species: timeseries(species, 'lat'), streams=[species_stream])
ts_speed = hv.DynamicMap(lambda species: timeseries(species, 'speed'), streams=[species_stream])

col = pn.Column('**Species**', species, ts_speed, ts_lat)
col

In [ ]:
print(col)

### Setting up the Table

We can get the air temp for a particular bird by selecting the `nearest` grid cell.

In [ ]:
def temp_calc(ds, row):
    lat_lon_day = row[['lat', 'lon', 'day']]
    return round(ds.sel(**lat_lon_day, method='nearest')['air_F'].item())

In [ ]:
temp_calc(ds, df.iloc[100])

Now we can set up a `holoviews.Table` element to report back the temperature. This function has a similar structure to the one above for calculating timeseries. 

In [ ]:
def daily_table(species=None, day=None):
    if not species or not day:
        return hv.Table(pd.DataFrame(columns=['Species', 'Air [F]', 'Speed [km/day]'])).relabel('No species selected')
    
    subset = df[df.species.isin(species)]
    subset = subset[subset.day==day]
    temps = [temp_calc(ds, row) for _, row in subset.iterrows()]
    
    return hv.Table(pd.DataFrame({'Species': species, 'Air [F]': temps, 'Speed [km/day]': subset['speed']})).relabel('day: {}'.format(day))

In [ ]:
daily_table().options(height=100)

In [ ]:
daily_table(['Veery'],5).options(height=100)

In [ ]:
table = hv.DynamicMap(daily_table, streams=[species_stream, day_stream])

### Hook up the map selector to the MultiSelect

The goal is to make it so that if you change the species selected on the map, then it will update species_stream. This will trigger the all the DynamicMaps that depend on species_stream to change as well.

In [ ]:
def on_map_select(index):
    if index:
        species = df.species.cat.categories[index].tolist()
        if set(species_stream.contents['species']) != set(species):
            species_stream.update(value=species)
        
map_selected_stream = Selection1D(source=bird_dmap)
map_selected_stream.param.watch_values(on_map_select, ['index']);

## Putting the pieces together

In [ ]:
dashboard = pn.Column(
    pn.Row('## Bird Migration Dashboard', pn.Spacer(width=200, height=80)),
    pn.Row(
        pn.Column(
            pn.Row(
                pn.Row(tiles * temp_layer * gv.feature.coastline * bird_dmap)[0][0], 
                pn.Spacer(width=20),
                pn.Column(
                    '**Day of Year**', day, 
                    '**Species**:',
                     'This selector does not affect the map. Use plot selectors.', species, 
                    toggle, highlight,
                    'This reset button only resets widgets - otherwise use the plot reset 🔄',
                    reset_button
                ),
                pn.Spacer(width=100),
            ),
            pn.Row(pn.layout.Tabs(('Latitude', ts_lat), ('Speed', ts_speed))),
        ),
        pn.Column(table.options(width=300, height=850))
    )
)

In [ ]:
dashboard.servable()

In [ ]:
print(dashboard)

Deploy this dashboard from the CLI using:

```
$ panel serve 04_panel.ipynb
```